## Question-2

In [1]:
!pip install pgmpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 15.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 25.2 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 MB 24.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.9/233.9 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 30.9 MB/s eta 0:00:00a 0:00:01


In [1]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination
import pandas as pd


In [2]:
file_path ='/Users/hoshea/Documents/pythonProject/Intro to AI/sample.txt'
def read_data(filepath):
    df = pd.read_csv(filepath)
    return df

if __name__ == "__main__": 
    df = read_data(file_path)
    
    # 打印前几行数据以验证
    print(df.head())

   smoker  treatment  dead
0       0          0     0
1       1          1     0
2       1          1     0
3       0          1     0
4       0          0     0


In [41]:
# (a)
# get the total number of patients
total_num = len(df)
# get the number and prob of smoker
num_smoker = len(df[df['smoker']==1])
prob_smoker = num_smoker/total_num

num_unsmoker = total_num - num_smoker
prob_unsmoker = num_unsmoker/total_num

# get the number and prob of treatment
num_treatment = len(df[df['treatment']==1])
prob_treatment = num_treatment/total_num

num_untreatment = total_num - num_treatment
prob_untreatment = num_untreatment/total_num

# get the number and prob of dead
num_dead = len(df[df['dead']==1])
prob_dead = num_dead/total_num

num_alive = total_num - num_dead
prob_alive = num_alive/total_num

# TE = Probability of being alive for the treated population – Probability of being alive for the untreated population.
# TE = P (dead = 0 , treatment = 1 )/P(treatment = 1) - P(dead = 0,treatment = 0)/P(treatment = 0)
#    = P (dead = 0 | treatment = 1) - P (dead = 0 | treatment = 0)

# get the number of alive people who get treated
num_treated_alive = len(df[(df['treatment'] == 1) & (df['dead'] == 0)])
cpd_treated_alive = num_treated_alive/num_treatment

# get the number of alive people who do not get treated
num_untreated_alive = len(df[(df['treatment'] == 0) & (df['dead'] == 0)])
cpd_untreated_alive = num_untreated_alive/num_untreatment

TE = cpd_treated_alive - cpd_untreated_alive
print('The TE only use the treatment and dead is:',TE)

The TE only use the treatment and dead is: -0.08484787082342737


In [42]:
num_smoker_treatment = len(df[(df['smoker'] == 1) & (df['treatment'] == 1)])
num_smoker_untreatment = len(df[(df['smoker'] == 1) & (df['treatment'] == 0)])
print('The number of smoker get treated ', num_smoker_treatment)
print('The number of smoker did not get treated ', num_smoker_untreatment)
print('The number of people get treated ', num_treatment)
print('The number of people not get treated ', num_untreatment)

The number of smoker get treated  165919
The number of smoker did not get treated  134904
The number of people get treated  305775
The number of people not get treated  694225


The TE dose not make sense because the total number of people not get treated is twice as people who get treated.
But the number of smokers get treated is larger than smokers not get treated.
And we all known that smoking can significantly increase the chance of dead.

In [63]:
# (b)
# build the BayesianNetwork
model = BayesianNetwork([
    ('Smoke', 'Treatment'),
    ('Treatment', 'Dead'),
    ('Smoke', 'Dead')
])

cpd_smoke = TabularCPD(variable='Smoke', variable_card=2, values=[[prob_unsmoker],[prob_smoker]])

# calculate the P(treatment|smoker)
num_unsmoker_untreatment = len(df[(df['smoker'] == 0) & (df['treatment'] == 0)])
num_unsmoker_treatment = len(df[(df['smoker'] == 0) & (df['treatment'] == 1)])
num_smoker_treatment = len(df[(df['smoker'] == 1) & (df['treatment'] == 1)])
num_smoker_untreatment = len(df[(df['smoker'] == 1) & (df['treatment'] == 0)])

prob_untreated_unsmoker = num_unsmoker_untreatment/num_unsmoker
prob_untreated_smoker = num_smoker_untreatment/num_smoker
prob_treated_unsmoker = num_unsmoker_treatment/num_unsmoker
prob_treated_smoker = num_smoker_treatment/num_smoker

cpd_treatment = TabularCPD(variable = 'Treatment',
                           variable_card=2,
                           values=[[prob_untreated_unsmoker,prob_untreated_smoker,],[prob_treated_unsmoker,prob_treated_smoker]],
                           evidence=['Smoke'],
                           evidence_card = [2])

# calculate the P(dead|smoker,treatment)
prob_dead_given_smoker_treatment = df.groupby(['smoker', 'treatment'])['dead'].mean().unstack().values
prob_alive_given_smoker_treatment = 1 - prob_dead_given_smoker_treatment

cpd_dead = TabularCPD(variable = 'Dead',
                           variable_card=2,
                           values=[
                               [prob_alive_given_smoker_treatment[0][0],
                                prob_alive_given_smoker_treatment[0][1],
                                prob_alive_given_smoker_treatment[1][0],
                                prob_alive_given_smoker_treatment[1][1]],
                               
                                [prob_dead_given_smoker_treatment[0][0],
                                 prob_dead_given_smoker_treatment[0][1],
                                 prob_dead_given_smoker_treatment[1][0],
                                 prob_dead_given_smoker_treatment[1][1]]
                           ],
                           evidence=['Smoke', 'Treatment'],
                           evidence_card = [2,2])

model.add_cpds(cpd_smoke,cpd_treatment,cpd_dead)

In [66]:
print(df.groupby(['smoker', 'treatment'])['dead'].mean().unstack())
print(df.groupby(['smoker', 'treatment'])['dead'].mean().unstack().values[0])

treatment         0         1
smoker                       
0          0.100125  0.111093
1          0.799124  0.497574
[0.10012497 0.11109284]


In [59]:
# Going to do variable elimination
infer = VariableElimination(model)

# compute conditional prob of alive
phi_query_treated = infer.query(['Dead'], evidence={'Treatment':1}, joint = False)
factor_treated = phi_query_treated['Dead']
phi_query_untreated = infer.query(['Dead'], evidence={'Treatment':0}, joint = False)
factor_untreated = phi_query_untreated['Dead']

In [60]:
print('The conditional prob of dead given treatment is :')
print(factor_treated)
print('The conditional prob of dead given untreatment is :')
print(factor_untreated)

The conditional prob of dead given treatment is :
+---------+-------------+
| Dead    |   phi(Dead) |
+=========+=============+
| Dead(0) |      0.6792 |
+---------+-------------+
| Dead(1) |      0.3208 |
+---------+-------------+
The conditional prob of dead given untreatment is :
+---------+-------------+
| Dead    |   phi(Dead) |
+=========+=============+
| Dead(0) |      0.7640 |
+---------+-------------+
| Dead(1) |      0.2360 |
+---------+-------------+


In [62]:
print('The TE of Bob"s idea is :',factor_treated.values[0]-factor_untreated.values[0])

The TE of Bob"s idea is : -0.08484787082342737


In [69]:
# (c)
model = BayesianNetwork([
    ('Smoke', 'Dead'),
    ('Treatment', 'Dead')
])

cpd_smoke = TabularCPD(variable='Smoke', variable_card=2, values=[[prob_unsmoker],[prob_smoker]])
cpd_treatment = TabularCPD(variable='Treatment', variable_card=2, values=[[prob_untreatment],[prob_treatment]])

cpd_dead = TabularCPD(variable = 'Dead',
                           variable_card=2,
                           values=[
                               [prob_alive_given_smoker_treatment[0][0],
                                prob_alive_given_smoker_treatment[0][1],
                                prob_alive_given_smoker_treatment[1][0],
                                prob_alive_given_smoker_treatment[1][1]],
                               
                                [prob_dead_given_smoker_treatment[0][0],
                                 prob_dead_given_smoker_treatment[0][1],
                                 prob_dead_given_smoker_treatment[1][0],
                                 prob_dead_given_smoker_treatment[1][1]]
                           ],
                           evidence=['Smoke', 'Treatment'],
                           evidence_card = [2,2])

model.add_cpds(cpd_smoke,cpd_treatment,cpd_dead)

In [48]:
[prob_untreatment],[prob_treatment]

([0.694225], [0.305775])

In [70]:
# Going to do variable elimination
infer = VariableElimination(model)

# compute conditional prob of alive
phi_query_treated = infer.query(['Dead'], evidence={'Treatment':1}, joint = False)
factor_treated = phi_query_treated['Dead']
phi_query_untreated = infer.query(['Dead'], evidence={'Treatment':0}, joint = False)
factor_untreated = phi_query_untreated['Dead']

print('The conditional prob of dead given treatment is :')
print(factor_treated)
print('The conditional prob of dead given untreatment is :')
print(factor_untreated)

The conditional prob of dead given treatment is :
+---------+-------------+
| Dead    |   phi(Dead) |
+=========+=============+
| Dead(0) |      0.7726 |
+---------+-------------+
| Dead(1) |      0.2274 |
+---------+-------------+
The conditional prob of dead given untreatment is :
+---------+-------------+
| Dead    |   phi(Dead) |
+=========+=============+
| Dead(0) |      0.6896 |
+---------+-------------+
| Dead(1) |      0.3104 |
+---------+-------------+


In [71]:
print('The TE of Alan"s idea is :',factor_treated.values[0]-factor_untreated.values[0])

The TE of Alan"s idea is : 0.08304460735461672


## Question-3

In [68]:
import numpy
#This class defines operators
class Operator:
    action = -1
    
    def __init__(self, act):
        self.action = act

#This class defines state 
class State:
    # This variable stores the tile configuration
    tiledArr = numpy.empty((3,3),dtype='int')
    
    # These two variables store the current location of empty tile
    freeTileI = 0
    freeTileJ = 0
    
    # This variable is to retrieve the path after finding a solution
    previous = None
    
    # This stores the cost from start node
    gVal = 0
    
    #Initializing the state with a given a tile configuration (arr)
    def __init__(self, arr):
        self.tiledArr = numpy.empty((3,3),dtype='int')
        for i in range(3):
            for j in range(3):
                self.tiledArr[i][j] = arr[i][j]
                
                #Empty tile is represented as a -1 in the 2D array
                if(self.tiledArr[i][j] == -1):
                    self.freeTileI = i
                    self.freeTileJ = j
    
    #This function returns a 1 if two tile configurations are the same and 0 otherwise
    def isEqual(self, s):
        for i in range(3):
            for j in range(3):
                if(self.tiledArr[i][j] != s.tiledArr[i][j]):
                    return 0
        return 1
    
    #This function applies an operator O on the current state (self) configuration 
    #and returns a new state configuration s1
    def applyOperator(self, o):
        s1 = State(self.tiledArr)
        s1.previous = self
        s1.gVal = self.gVal + 1
        #0 - West, 1 - South, 2 - East and 3 - North'
        if(o.action == 0):
            if(self.freeTileJ - 1 >= 0):
                s1.tiledArr[self.freeTileI][self.freeTileJ] = self.tiledArr[self.freeTileI][self.freeTileJ-1]
                s1.freeTileJ -= 1
        elif(o.action == 1):
            if(self.freeTileI + 1 < 3):
                s1.tiledArr[self.freeTileI][self.freeTileJ] = self.tiledArr[self.freeTileI+1][self.freeTileJ]
                s1.freeTileI += 1
        elif(o.action == 2):
            if(self.freeTileJ + 1 < 3):
                s1.tiledArr[self.freeTileI][self.freeTileJ] = self.tiledArr[self.freeTileI][self.freeTileJ+1]
                s1.freeTileJ += 1
        elif(o.action == 3):
            if(self.freeTileI - 1 >= 0):
                s1.tiledArr[self.freeTileI][self.freeTileJ] = self.tiledArr[self.freeTileI-1][self.freeTileJ]
                s1.freeTileI -= 1
        
        s1.tiledArr[s1.freeTileI][s1.freeTileJ] = -1
                
        return s1
    
    #This function prints the state configuration
    def printState(self):
        for i in range(3):
            for j in range(3):
                print("%d"%self.tiledArr[i][j],end=' ')
            print()
        print("*******")

In [72]:
#Function that indicates whether a state is present in a list
def isPresentStateInList(state, searchList):
    for elem in searchList:
        if(state.isEqual(elem) == 1):
            return 1

    return 0

#Function that indicates whether a state is present in a priority list
def isPresentStateInPriorityList(state, searchList):
    for [elem, dist] in searchList:
        if(state.isEqual(elem) == 1):
            return 1
        
    return 0

def insertStateInPriorityQueue(searchList, state, distanceToGoal):
    index = -1
    for i in range(len(searchList)):
        if(distanceToGoal < searchList[i][1]):
            index = i
            break
    
    if(len(searchList) == 0 or index == -1):
        searchList.append([state,distanceToGoal])
    else:
        searchList.insert(index, [state,distanceToGoal])

#Reinserts the element in priority queue if the new value is less than the 
#value present in queue.
def checkAndUpdateStateInPriorityQueue(searchList,state,distanceToGoal):
    index = -1
    for i in range(len(searchList)):
        if(state.isEqual(searchList[i][0])):
            if distanceToGoal < searchList[i][1]:
                    index = i
            break
            
    if index!=-1:
        searchList.remove([searchList[index][0],searchList[index][1]])
        insertStateInPriorityQueue(searchList, state, distanceToGoal)
        

        
        
def retrievePathFromState(state):
    visitedstatelist = []
    visitedstatelist.append(state)
    state.printState()
    prev = state.previous
    counter = 0
    while(prev != None):
        visitedstatelist.append(prev)
        #prev.printState()
        prev = prev.previous
        counter +=1
    visitedstatelist.reverse()
    for i in range(len(visitedstatelist)):
        visitedstatelist[i].printState()
    print("Size of path is ",counter,state.gVal)

In [73]:
import math
# Hamming distance
def calculateHamming(current, goal):
    counter = 0
    for i in range(3):
        for j in range(3):
            if(current.tiledArr[i][j] != goal.tiledArr[i][j]):
                counter+=1

    return counter

# Manhattan distance
def calculateManhattan(current, goal):
    xposcurrent = [-1,-1,-1,-1,-1,-1,-1,-1,-1]
    yposcurrent = [-1,-1,-1,-1,-1,-1,-1,-1,-1]
    xposgoal =  [-1,-1,-1,-1,-1,-1,-1,-1,-1]
    yposgoal =  [-1,-1,-1,-1,-1,-1,-1,-1,-1]
    for i in range(3):
        for j in range(3):
            if current.tiledArr[i][j]!=-1:
                xposcurrent[current.tiledArr[i][j]] = i
                yposcurrent[current.tiledArr[i][j]] = j
            if goal.tiledArr[i][j]!=-1:
                xposgoal[goal.tiledArr[i][j]] = i
                yposgoal[goal.tiledArr[i][j]] = j
    totdist=0
    for i in range(1,9):
        totdist+= (math.fabs(xposcurrent[i]-xposgoal[i])+math.fabs(yposcurrent[i]-yposgoal[i]))
    
    return totdist

In [79]:
# (a)
def calculateXDistance(current, goal):
    row_count = 0
    column_count = 0
    for i in range(3):
        for j in range(3):
            if current.tiledArr[i][j] != -1:
                if current.tiledArr[i][j] != goal.tiledArr[i][j]:
                    goal_i, goal_j = numpy.where(goal.tiledArr == current.tiledArr[i][j])
                    if goal_i[0] != i:
                        row_count += 1
                    if goal_j[0] != j:
                        column_count += 1
    return row_count + column_count

def a_star(initial_state, goal_state, heuristic):
    open_list = []
    closed_list = []
    open_list.append((initial_state, heuristic(initial_state, goal_state)))
    node_reopen_count = 0

    while open_list:
        current_state, current_cost = open_list.pop(0)
        
        if current_state.isEqual(goal_state):
            retrievePathFromState(current_state)
            print(f"Node reopen count: {node_reopen_count}")
            return current_state.gVal, node_reopen_count
        
        closed_list.append(current_state)
        
        for action in range(4):
            operator = Operator(action)
            next_state = current_state.applyOperator(operator)
            
            if isPresentStateInList(next_state, closed_list):
                continue
            
            next_cost = next_state.gVal + heuristic(next_state, goal_state)
            
            if isPresentStateInPriorityList(next_state, open_list):
                checkAndUpdateStateInPriorityQueue(open_list, next_state, next_cost)
                node_reopen_count += 1
            else:
                insertStateInPriorityQueue(open_list, next_state, next_cost)
    
    return None, node_reopen_count

In [80]:
# case 1
initial_arr = numpy.array([[8,7,1], [6,-1,2], [5,4,3]])
goal_arr = numpy.array([[1, 2, 3], [4,5,6], [7,8,-1]])

initial_state = State(initial_arr)
goal_state = State(goal_arr)

# Running A* with Hamming distance
gVal, reopen_count = a_star(initial_state, goal_state, calculateHamming)
print(f"Hamming distance - Cost: {gVal}, Node reopens: {reopen_count}")

# Running A* with Manhattan distance
gVal, reopen_count = a_star(initial_state, goal_state, calculateManhattan)
print(f"Manhattan distance - Cost: {gVal}, Node reopens: {reopen_count}")

# Running A* with X-distance
gVal, reopen_count = a_star(initial_state, goal_state, calculateXDistance)
print(f"X-distance - Cost: {gVal}, Node reopens: {reopen_count}")

1 2 3 
4 5 6 
7 8 -1 
*******
8 7 1 
6 -1 2 
5 4 3 
*******
8 7 1 
6 4 2 
5 -1 3 
*******
8 7 1 
6 4 2 
-1 5 3 
*******
8 7 1 
-1 4 2 
6 5 3 
*******
-1 7 1 
8 4 2 
6 5 3 
*******
7 -1 1 
8 4 2 
6 5 3 
*******
7 1 -1 
8 4 2 
6 5 3 
*******
7 1 2 
8 4 -1 
6 5 3 
*******
7 1 2 
8 4 3 
6 5 -1 
*******
7 1 2 
8 4 3 
6 -1 5 
*******
7 1 2 
8 4 3 
-1 6 5 
*******
7 1 2 
-1 4 3 
8 6 5 
*******
-1 1 2 
7 4 3 
8 6 5 
*******
1 -1 2 
7 4 3 
8 6 5 
*******
1 2 -1 
7 4 3 
8 6 5 
*******
1 2 3 
7 4 -1 
8 6 5 
*******
1 2 3 
7 4 5 
8 6 -1 
*******
1 2 3 
7 4 5 
8 -1 6 
*******
1 2 3 
7 4 5 
-1 8 6 
*******
1 2 3 
-1 4 5 
7 8 6 
*******
1 2 3 
4 -1 5 
7 8 6 
*******
1 2 3 
4 5 -1 
7 8 6 
*******
1 2 3 
4 5 6 
7 8 -1 
*******
Size of path is  22 22
Node reopen count: 861
Hamming distance - Cost: 22, Node reopens: 861
1 2 3 
4 5 6 
7 8 -1 
*******
8 7 1 
6 -1 2 
5 4 3 
*******
8 7 1 
-1 6 2 
5 4 3 
*******
8 7 1 
5 6 2 
-1 4 3 
*******
8 7 1 
5 6 2 
4 -1 3 
*******
8 7 1 
5 -1 2 
4 6 3 
*******
8 -1 1 

In [81]:
# case 2
initial_arr = numpy.array([[7,2,4], [5,-1,6], [8,3,1]])
goal_arr = numpy.array([[-1, 1, 2], [3,4,5], [6,7,8]])

initial_state = State(initial_arr)
goal_state = State(goal_arr)

# Running A* with Hamming distance
gVal, reopen_count = a_star(initial_state, goal_state, calculateHamming)
print(f"Hamming distance - Cost: {gVal}, Node reopens: {reopen_count}")

# Running A* with Manhattan distance
gVal, reopen_count = a_star(initial_state, goal_state, calculateManhattan)
print(f"Manhattan distance - Cost: {gVal}, Node reopens: {reopen_count}")

# Running A* with X-distance
gVal, reopen_count = a_star(initial_state, goal_state, calculateXDistance)
print(f"X-distance - Cost: {gVal}, Node reopens: {reopen_count}")

-1 1 2 
3 4 5 
6 7 8 
*******
7 2 4 
5 -1 6 
8 3 1 
*******
7 2 4 
-1 5 6 
8 3 1 
*******
-1 2 4 
7 5 6 
8 3 1 
*******
2 -1 4 
7 5 6 
8 3 1 
*******
2 5 4 
7 -1 6 
8 3 1 
*******
2 5 4 
7 6 -1 
8 3 1 
*******
2 5 4 
7 6 1 
8 3 -1 
*******
2 5 4 
7 6 1 
8 -1 3 
*******
2 5 4 
7 6 1 
-1 8 3 
*******
2 5 4 
-1 6 1 
7 8 3 
*******
2 5 4 
6 -1 1 
7 8 3 
*******
2 5 4 
6 1 -1 
7 8 3 
*******
2 5 4 
6 1 3 
7 8 -1 
*******
2 5 4 
6 1 3 
7 -1 8 
*******
2 5 4 
6 1 3 
-1 7 8 
*******
2 5 4 
-1 1 3 
6 7 8 
*******
2 5 4 
1 -1 3 
6 7 8 
*******
2 5 4 
1 3 -1 
6 7 8 
*******
2 5 -1 
1 3 4 
6 7 8 
*******
2 -1 5 
1 3 4 
6 7 8 
*******
-1 2 5 
1 3 4 
6 7 8 
*******
1 2 5 
-1 3 4 
6 7 8 
*******
1 2 5 
3 -1 4 
6 7 8 
*******
1 2 5 
3 4 -1 
6 7 8 
*******
1 2 -1 
3 4 5 
6 7 8 
*******
1 -1 2 
3 4 5 
6 7 8 
*******
-1 1 2 
3 4 5 
6 7 8 
*******
Size of path is  26 26
Node reopen count: 8251
Hamming distance - Cost: 26, Node reopens: 8251
-1 1 2 
3 4 5 
6 7 8 
*******
7 2 4 
5 -1 6 
8 3 1 
*******
7 2 4

In [82]:
# case 3
initial_arr = numpy.array([[7, 2, 4], [5, -1, 6], [8, 3, 1]])
goal_arr = numpy.array([[1, 6, 7], [2, 5, 8], [3, 4, -1]])

initial_state = State(initial_arr)
goal_state = State(goal_arr)

# Running A* with Hamming distance
gVal, reopen_count = a_star(initial_state, goal_state, calculateHamming)
print(f"Hamming distance - Cost: {gVal}, Node reopens: {reopen_count}")

# Running A* with Manhattan distance
gVal, reopen_count = a_star(initial_state, goal_state, calculateManhattan)
print(f"Manhattan distance - Cost: {gVal}, Node reopens: {reopen_count}")

# Running A* with X-distance
gVal, reopen_count = a_star(initial_state, goal_state, calculateXDistance)
print(f"X-distance - Cost: {gVal}, Node reopens: {reopen_count}")

1 6 7 
2 5 8 
3 4 -1 
*******
7 2 4 
5 -1 6 
8 3 1 
*******
7 -1 4 
5 2 6 
8 3 1 
*******
-1 7 4 
5 2 6 
8 3 1 
*******
5 7 4 
-1 2 6 
8 3 1 
*******
5 7 4 
2 -1 6 
8 3 1 
*******
5 7 4 
2 6 -1 
8 3 1 
*******
5 7 4 
2 6 1 
8 3 -1 
*******
5 7 4 
2 6 1 
8 -1 3 
*******
5 7 4 
2 -1 1 
8 6 3 
*******
5 7 4 
2 1 -1 
8 6 3 
*******
5 7 -1 
2 1 4 
8 6 3 
*******
5 -1 7 
2 1 4 
8 6 3 
*******
5 1 7 
2 -1 4 
8 6 3 
*******
5 1 7 
2 6 4 
8 -1 3 
*******
5 1 7 
2 6 4 
-1 8 3 
*******
5 1 7 
-1 6 4 
2 8 3 
*******
-1 1 7 
5 6 4 
2 8 3 
*******
1 -1 7 
5 6 4 
2 8 3 
*******
1 6 7 
5 -1 4 
2 8 3 
*******
1 6 7 
5 8 4 
2 -1 3 
*******
1 6 7 
5 8 4 
2 3 -1 
*******
1 6 7 
5 8 -1 
2 3 4 
*******
1 6 7 
5 -1 8 
2 3 4 
*******
1 6 7 
-1 5 8 
2 3 4 
*******
1 6 7 
2 5 8 
-1 3 4 
*******
1 6 7 
2 5 8 
3 -1 4 
*******
1 6 7 
2 5 8 
3 4 -1 
*******
Size of path is  26 26
Node reopen count: 8235
Hamming distance - Cost: 26, Node reopens: 8235
1 6 7 
2 5 8 
3 4 -1 
*******
7 2 4 
5 -1 6 
8 3 1 
*******
7 -1 